In [ ]:
import os

def scale_input_data(scale_factor):
  file_bases = ['./input/vgsales']
  for file_base in file_bases:
    import pandas as pd
    import shutil
    if scale_factor == 1.0:
      shutil.copyfile(file_base + '.csv', file_base + '.scaled.csv')
      continue
    df_to_scale = pd.read_csv(file_base + '.csv')
    new_num_rows = int(scale_factor * len(df_to_scale))
    if scale_factor <= 1.0:
      df_to_scale = df_to_scale.iloc[:new_num_rows]
    else:
      while len(df_to_scale) < new_num_rows:
        df_to_scale = pd.concat([df_to_scale, df_to_scale[:min(new_num_rows - len(df_to_scale), len(df_to_scale))]])
    df_to_scale.to_csv(file_base + '.scaled.csv', index=False)

if 'INPUT_SCALE_FACTOR' in os.environ:
  scale_input_data(float(os.environ['INPUT_SCALE_FACTOR']))

---

<h1 style="text-align: center;font-size: 40px;">Video Games Sales Analysis And Visualization</h1>

---

<center><img src="https://i.pinimg.com/originals/68/f3/ff/68f3ff8ddc1699f6234abee4e1d58dd9.gif"></center>

---
<i>image from Google</i>

<h4> Story is that, one of my roommate "Jaber" who playing video games too much and love it much also. One day he came into my room and asked me about some stat and figure about video games, i said bro i dont know much about video games sorry for that but i can help you another way, that i know a good data sate which have much info about games. </h4>
<h4>so i can show you this data set and you ask me whatever you want to know i will try to find out. ok?</h4>

<h4>He was very happy on my offer. and he just starting many question at once. i said wait bro i need to go import them first to start analysis. be patient </h4>

In [1]:
import numpy as np
# import pandas as pd
exec(os.environ['IREWR_IMPORTS'])
# FIRST-AUTHOR: remove ML code
# import scipy.stats as st
pd.set_option('display.max_columns', None)

import math

# FIRST-AUTHOR: remove plotting, ML code
# import matplotlib.pyplot as plt
# %matplotlib inline

# import seaborn as sns
# sns.set_style('whitegrid')

# import missingno as msno

# from sklearn.preprocessing import StandardScaler
# from scipy import stats


# FIRST-AUTHOR: remove path printing
# import os
# for dirname, _, filenames in os.walk('./input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [2]:
data = pd.read_csv('./input/vgsales.scaled.csv')
data.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


Me => See, here is your video games data.

In [3]:
drop_row_index = data[data['Year'] > 2015].index
data = data.drop(drop_row_index)

* Me => Deleted some incomplete data. You can see, from 2016 the data we have those are not fully completed.
* Jaber => Is it necessary?
* Me => Yes, it will help us to do better analysis. Incomplete data always hamper in analysis.

In [4]:
data.shape

(16250, 11)

* Me => total 11 columns and 16598 rows. which is actually good enough.
* Jber => what does that mean of the 11 columns. What EU_Sales, JP_Sales means, i don't understand all of them.
* Me => Ok i will tell you about them with in a second.

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16250 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rank          16250 non-null  int64  
 1   Name          16250 non-null  object 
 2   Platform      16250 non-null  object 
 3   Year          15979 non-null  float64
 4   Genre         16250 non-null  object 
 5   Publisher     16194 non-null  object 
 6   NA_Sales      16250 non-null  float64
 7   EU_Sales      16250 non-null  float64
 8   JP_Sales      16250 non-null  float64
 9   Other_Sales   16250 non-null  float64
 10  Global_Sales  16250 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 1.5+ MB


**Their fields and data types are:**

   * Rank - Ranking of overall sales, integer
   
   * Name - The games name

   * Platform - Platform of the games release (i.e. PC,PS4, etc.), object

   * Year - Year of the game's release, float

   * Genre - Genre of the game ,object

   * Publisher - Publisher of the game, object

   * NA_Sales - Sales in North America (in millions), float

   * EU_Sales - Sales in Europe (in millions), float

   * JP_Sales - Sales in Japan (in millions), float

   * Other_Sales - Sales in the rest of the world (in millions), float

   * Global_Sales - Total worldwide sales, float



* Jaber => Now i understand meaning of columns names but why you told me their types 🤔. 
* Me => Ha ha, its not for you, its for me. to understand what kind of data types i get for analysis. and i see most of them are float and integer which help me lot.
* Jaber => 🙄 🙄

In [6]:
# data.describe()

In [7]:
# data.describe(include=['object', 'bool'])

In [8]:
data.isnull().sum()

Rank              0
Name              0
Platform          0
Year            271
Genre             0
Publisher        56
NA_Sales          0
EU_Sales          0
JP_Sales          0
Other_Sales       0
Global_Sales      0
dtype: int64

Me => Happy news for you Jaber, we don't have much missing values. that means we are not losing much information. see i told you it's already good readymade dataset.

* Jaber => 😡😡 Why you told me too many unnecessary information, When can I ask my question?
* Me => Hmmm hmm, Just Now.

He started his unstoppable questions bank for me, here it's started.

# 1. What genre games have been made the most?

In [9]:
data['Genre'].value_counts()

Action          3196
Sports          2308
Misc            1721
Role-Playing    1446
Shooter         1278
Adventure       1252
Racing          1229
Platform         876
Simulation       857
Fighting         834
Strategy         671
Puzzle           582
Name: Genre, dtype: int64

In [10]:
# FIRST-AUTHOR: remove plotting
# plt.figure(figsize=(15, 10))
# sns.countplot(x="Genre", data=data, order = data['Genre'].value_counts().index)
# plt.xticks(rotation=90)
_ = data['Genre'].value_counts().index

### Answer is => "Action" and "Sports"
* Me => I see, most of the people love action and sports game. in action 3316 and in sports 2346 games release. which one your favorite.
* Jaber => Action 
* Me => I thought so too, Next?

# 2. Which year had the most game release?

In [11]:
# FIRST-AUTHOR: remove plotting
# plt.figure(figsize=(15, 10))
# sns.countplot(x="Year", data=data, order = data.groupby(by=['Year'])['Name'].count().sort_values(ascending=False).index)
# plt.xticks(rotation=90)
_ = data.groupby(by=['Year'])['Name'].count().sort_values(ascending=False).index

### Answer is =>
1. 2009.0  ->  1431
1. 2008.0  ->  1428
1. 2010.0  ->  1259
1. 2007.0  ->  1202
1. 2011.0  ->  1139

Me => its seems like that 2008 to 2010 was game lovers years. and you know jaber, i can show you one interesting things also.

# 3. Top 5 years games release by genre.

In [12]:
# FIRST-AUTHOR: remove plotting
# plt.figure(figsize=(30, 10))
# sns.countplot(x="Year", data=data, hue='Genre', order=data.Year.value_counts().iloc[:5].index)
# plt.xticks(size=16, rotation=90)
_ = data.Year.value_counts().iloc[:5].index

* Me => ??
* Jaber => Wow thats nice and action was always top.
* Me => Ask your next question? i'm in mood now.

# 4. Which year had the highest sales worldwide?

In [13]:
data_year = data.groupby(by=['Year'])['Global_Sales'].sum()
data_year = data_year.reset_index()
# data_year.sort_values(by=['Global_Sales'], ascending=False)

In [14]:
# FIRST-AUTHOR: remove plotting
# plt.figure(figsize=(15, 10))
# sns.barplot(x="Year", y="Global_Sales", data=data_year)
# plt.xticks(rotation=90)

### Answer is =>
1. 2008.0 	678.90
1. 2009.0 	667.30
1. 2007.0 	611.13
1. 2010.0 	600.45
1. 2006.0 	521.04

* interesting 2006 was not in top 5 game release year but it is in top 5 highest sale year

# 5. Which genre game has been released the most in a single year?

In [15]:
year_max_df = data.groupby(['Year', 'Genre']).size().reset_index(name='count')
year_max_idx = year_max_df.groupby(['Year'])['count'].transform(max) == year_max_df['count']
year_max_genre = year_max_df[year_max_idx].reset_index(drop=True)
year_max_genre = year_max_genre.drop_duplicates(subset=["Year", "count"], keep='last').reset_index(drop=True)
# year_max_genre

In [16]:
genre = year_max_genre['Genre'].values
# genre[0]

In [17]:
# FIRST-AUTHOR: remove plotting
# plt.figure(figsize=(30, 15))
# g = sns.barplot(x='Year', y='count', data=year_max_genre)
index = 0
for value in year_max_genre['count'].values:
#     print(asd)
# FIRST-AUTHOR: remove plotting
#     g.text(index, value + 5, str(genre[index] + '----' +str(value)), color='#000', size=14, rotation= 90, ha="center")
    _ = str(genre[index] + '----' +str(value))
    index += 1




# FIRST-AUTHOR: remove plotting
# plt.xticks(rotation=90)
# plt.show()

### Answer is => 2009 Action ---> 272 and 2012 Action ---> 266 games are release.

# 6. Which genre game has sold the most in a single year?

In [18]:
year_sale_dx = data.groupby(by=['Year', 'Genre'])['Global_Sales'].sum().reset_index()
year_sale = year_sale_dx.groupby(by=['Year'])['Global_Sales'].transform(max) == year_sale_dx['Global_Sales']
year_sale_max = year_sale_dx[year_sale].reset_index(drop=True)
# year_sale_max

In [19]:
genre = year_sale_max['Genre']

In [20]:
# FIRST-AUTHOR: remove plotting
# plt.figure(figsize=(30, 18))
# g = sns.barplot(x='Year', y='Global_Sales', data=year_sale_max)
index = 0
for value in year_sale_max['Global_Sales']:
# FIRST-AUTHOR: remove plotting
#     g.text(index, value + 1, str(genre[index] + '----' +str(round(value, 2))), color='#000', size=14, rotation= 90, ha="center")
    _ = str(genre[index] + '----' +str(round(value, 2)))
    index += 1

# FIRST-AUTHOR: remove plotting
# plt.xticks(rotation=90)
# plt.show()

* Answer is => 2009 Action ---> 139.36 million and 2008 Action ---> 136.39 miliion .

# 7. Which genre game have the highest sale price globally

In [21]:
data_genre = data.groupby(by=['Genre'])['Global_Sales'].sum()
data_genre = data_genre.reset_index()
data_genre = data_genre.sort_values(by=['Global_Sales'], ascending=False)
# data_genre

In [22]:
# FIRST-AUTHOR: remove plotting
# plt.figure(figsize=(15, 10))
# sns.barplot(x="Genre", y="Global_Sales", data=data_genre)
# plt.xticks(rotation=90)

* Action and Sports are always in top. but in third whis is remarkable "Shooter", in count which was no 5

# 8. Which platfrom have the highest sale price globally

In [23]:
data_platform = data.groupby(by=['Platform'])['Global_Sales'].sum()
data_platform = data_platform.reset_index()
data_platform = data_platform.sort_values(by=['Global_Sales'], ascending=False)
# data_platform

In [24]:
# FIRST-AUTHOR: remove plotting
# plt.figure(figsize=(15, 10))
# sns.barplot(x="Platform", y="Global_Sales", data=data_platform)
# plt.xticks(rotation=90)

* **The winner is PS2**

# 9. Which individual game have the highest sale price globally?

In [25]:
top_game_sale = data.head(20)
top_game_sale = top_game_sale[['Name', 'Year', 'Genre', 'Global_Sales']]
top_game_sale = top_game_sale.sort_values(by=['Global_Sales'], ascending=False)
# top_game_sale

In [26]:
name = top_game_sale['Name']
year = top_game_sale['Year']
y = np.arange(0, 20)

In [27]:
# FIRST-AUTHOR: remove plotting
# plt.figure(figsize=(30, 18))
# g = sns.barplot(x='Name', y='Global_Sales', data=top_game_sale)
index = 0
for value in top_game_sale['Global_Sales']:
# FIRST-AUTHOR: remove plotting
#     g.text(index, value - 18, name[index], color='#000', size=14, rotation= 90, ha="center")
    _ = name[index]
    index += 1

# FIRST-AUTHOR: remove plotting
# plt.xticks(y, top_game_sale['Year'], fontsize=14, rotation=90)
# plt.xlabel('Release Year')
# plt.show()
_ = top_game_sale['Year']

* **And The winner is Wii Sports**

ME => Ok jaber i thought you asked me too many questions. Can you hold it for time being, i will show you something interesting
Jaber => Ok, i have no problem let's start

# 10. Sales compearison by genre

In [28]:
comp_genre = data[['Genre', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']]
# comp_genre
comp_map = comp_genre.groupby(by=['Genre']).sum()
# comp_map

In [29]:
# FIRST-AUTHOR: remove plotting
# plt.figure(figsize=(15, 10))
# sns.set(font_scale=1)
# sns.heatmap(comp_map, annot=True, fmt = '.1f')

# plt.xticks(fontsize=14)
# plt.yticks(fontsize=14)
# plt.show()

In [30]:
comp_table = comp_map.reset_index()
comp_table = pd.melt(comp_table, id_vars=['Genre'], value_vars=['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales'], var_name='Sale_Area', value_name='Sale_Price')
comp_table.head()

,Genre,Sale_Area,Sale_Price
0,Action,NA_Sales,871.96
1,Adventure,NA_Sales,105.46
2,Fighting,NA_Sales,221.99
3,Misc,NA_Sales,410.02
4,Platform,NA_Sales,446.26


In [31]:
# FIRST-AUTHOR: remove plotting
# plt.figure(figsize=(15, 10))
# sns.barplot(x='Genre', y='Sale_Price', hue='Sale_Area', data=comp_table)

Me => See, Here Action Sports and Shooter have much sales compare with others. and North America (NA_sales) have heights sales all the time. Its look like they love video games most

# 11. Sales compearison by platform

In [32]:
comp_platform = data[['Platform', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']]
comp_platform.head()

,Platform,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,Wii,41.49,29.02,3.77,8.46
1,NES,29.08,3.58,6.81,0.77
2,Wii,15.85,12.88,3.79,3.31
3,Wii,15.75,11.01,3.28,2.96
4,GB,11.27,8.89,10.22,1.00


In [33]:
comp_platform = comp_platform.groupby(by=['Platform']).sum().reset_index()

In [34]:
# comp_table = comp_map.reset_index()
comp_table = pd.melt(comp_platform, id_vars=['Platform'], value_vars=['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales'], var_name='Sale_Area', value_name='Sale_Price')
comp_table.head()

,Platform,Sale_Area,Sale_Price
0,2600,NA_Sales,90.60
1,3DO,NA_Sales,0.00
2,3DS,NA_Sales,78.04
3,DC,NA_Sales,5.43
4,DS,NA_Sales,390.44


In [35]:
# FIRST-AUTHOR: remove plotting
# plt.figure(figsize=(30, 15))
# sns.barplot(x='Platform', y='Sale_Price', hue='Sale_Area', data=comp_table)
# plt.xticks(fontsize=14)
# plt.yticks(fontsize=14)
# plt.show()

ME => It's clearly shown that X360, Wii, and PS are market leader. and north america always top. i have a question in my mind why they play video games most?

# 12. Top 20 Publisher

In [36]:
top_publisher = data.groupby(by=['Publisher'])['Year'].count().sort_values(ascending=False).head(20)
top_publisher = pd.DataFrame(top_publisher).reset_index()
# top_publisher

In [37]:
# FIRST-AUTHOR: remove plotting
# plt.figure(figsize=(15, 10))
# sns.countplot(x="Publisher", data=data, order = data.groupby(by=['Publisher'])['Year'].count().sort_values(ascending=False).iloc[:20].index)
# plt.xticks(rotation=90)
_ = data.groupby(by=['Publisher'])['Year'].count().sort_values(ascending=False).iloc[:20].index

* Jaber => Electronic Arts 1339, its huge
* Me => Thank you EA Sports for giving us too many games.

# 13. Top global sales by publisher

In [38]:
sale_pbl = data[['Publisher', 'Global_Sales']]
sale_pbl = sale_pbl.groupby('Publisher')['Global_Sales'].sum().sort_values(ascending=False).head(20)
sale_pbl = pd.DataFrame(sale_pbl).reset_index()
# sale_pbl

In [39]:
# FIRST-AUTHOR: remove plotting
# plt.figure(figsize=(15, 10))
# sns.barplot(x='Publisher', y='Global_Sales', data=sale_pbl)
# plt.xticks(rotation=90)

1. ME => Wow its awesome Nintendo earns more rather than others. where respectively they are not releases much games.
1. Jaber => Till now Nintendo release 696 games. are you dont think its not huge?
1. Me => Yeah maybe, but they are not top 5 publisher

# 14. publisher comperison

In [40]:
comp_publisher = data[['Publisher', 'NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']]
comp_publisher.head()

,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,Nintendo,41.49,29.02,3.77,8.46,82.74
1,Nintendo,29.08,3.58,6.81,0.77,40.24
2,Nintendo,15.85,12.88,3.79,3.31,35.82
3,Nintendo,15.75,11.01,3.28,2.96,33.00
4,Nintendo,11.27,8.89,10.22,1.00,31.37


In [41]:
comp_publisher = comp_publisher.groupby(by=['Publisher']).sum().reset_index().sort_values(by=['Global_Sales'], ascending=False)
comp_publisher = comp_publisher.head(20)
# comp_publisher

In [42]:
comp_publisher = pd.melt(comp_publisher, id_vars=['Publisher'], value_vars=['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales'], var_name='Sale_Area', value_name='Sale_Price')
comp_publisher

,Publisher,Sale_Area,Sale_Price
0,Nintendo,NA_Sales,815.71
1,Electronic Arts,NA_Sales,591.43
2,Activision,NA_Sales,428.03
3,Sony Computer Entertainment,NA_Sales,262.79
4,Ubisoft,NA_Sales,249.31
...,...,...,...
75,Disney Interactive Studios,Other_Sales,13.18
76,Eidos Interactive,Other_Sales,8.02
77,LucasArts,Other_Sales,10.50
78,Bethesda Softworks,Other_Sales,9.87


In [43]:
# FIRST-AUTHOR: remove plotting
# plt.figure(figsize=(30, 15))
# sns.barplot(x='Publisher', y='Sale_Price', hue='Sale_Area', data=comp_publisher)
# plt.xticks(fontsize=14, rotation=90)
# plt.yticks(fontsize=14)
# plt.show()

# 15. Top publisher by Count each year

In [44]:
top_publisher =  data[['Year', 'Publisher']]
top_publisher_df = top_publisher.groupby(by=['Year', 'Publisher']).size().reset_index(name='Count')
top_publisher_idx =  top_publisher_df.groupby(by=['Year'])['Count'].transform(max) == top_publisher_df['Count']
top_publisher_count = top_publisher_df[top_publisher_idx].reset_index(drop=True)
top_publisher_count  = top_publisher_count.drop_duplicates(subset=["Year", "Count"], keep='last').reset_index(drop=True)
# top_publisher_count

In [45]:
publisher= top_publisher_count['Publisher']

In [46]:
# FIRST-AUTHOR: remove plotting
# plt.figure(figsize=(30, 15))
# g = sns.barplot(x='Year', y='Count', data=top_publisher_count)
index = 0
for value in top_publisher_count['Count'].values:
#     print(asd)
# FIRST-AUTHOR: remove plotting
#     g.text(index, value + 5, str(publisher[index] + '----' +str(value)), color='#000', size=14, rotation= 90, ha="center")
    _ = str(publisher[index] + '----' +str(value))
    index += 1




# FIRST-AUTHOR: remove plotting
# plt.xticks(rotation=90)
# plt.show()

# 16. Total revenue by region

In [47]:
# data.head()

In [48]:
top_sale_reg = data[['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']]
# pd.DataFrame(top_sale_reg.sum(), columns=['a', 'b'])
top_sale_reg = top_sale_reg.sum().reset_index()
top_sale_reg = top_sale_reg.rename(columns={"index": "region", 0: "sale"})
top_sale_reg

,region,sale
0,NA_Sales,4370.02
1,EU_Sales,2407.37
2,JP_Sales,1277.27
3,Other_Sales,789.98


In [49]:
# FIRST-AUTHOR: remove plotting
# plt.figure(figsize=(12, 8))
# sns.barplot(x='region', y='sale', data = top_sale_reg)

### pie chart of region sale

In [50]:
labels = top_sale_reg['region']
sizes = top_sale_reg['sale']

In [51]:
# FIRST-AUTHOR: remove plotting
# plt.figure(figsize=(10, 8))
# plt.pie(sizes, labels=labels, autopct='%1.1f%%', shadow=True, startangle=90)

North America almost Cover 50% of sales.

# 17. Sales Histogram

In [52]:
# sns.distplot(data['NA_Sales'],  kde=False, fit=stats.gamma);
# sns.distplot(data['EU_Sales'],  kde=False, fit=stats.gamma);
# FIRST-AUTHOR: remove plotting
# plt.figure(figsize=(25,30))
sales_columns = ['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']
for i, column in enumerate(sales_columns):
# FIRST-AUTHOR: remove plotting
#     plt.subplot(3,2,i+1)
#     sns.distplot(data[column], bins=20, kde=False, fit=stats.gamma)
    _ = data[column]

## Distribution After log transformation

In [53]:
data_hist_log = data.copy()

In [54]:
data_hist_log = data_hist_log[data_hist_log.NA_Sales != 0]
data_hist_log = data_hist_log[data_hist_log.EU_Sales != 0]
data_hist_log = data_hist_log[data_hist_log.Other_Sales != 0]
data_hist_log = data_hist_log[data_hist_log.JP_Sales != 0]
data_hist_log = data_hist_log[data_hist_log.Global_Sales != 0]

In [55]:
# FIRST-AUTHOR: remove plotting
# plt.figure(figsize=(25,30))
sales_columns = ['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales']
for i, column in enumerate(sales_columns):
# FIRST-AUTHOR: remove plotting
#     plt.subplot(3,2,i+1)
#     sns.distplot(np.log(data_hist_log[column]), bins=20, kde=False, fit=stats.gamma)
    _ = np.log(data_hist_log[column])

You can see their skewness and central tendency. we can say that our almost all data are right skewed and symmetric skewed after log transformation

<img src="https://miro.medium.com/max/3164/1*pTe0p8HMmEnbAn8uEL66Ng.png" />

# 18. Relations of columns

## heatmap

**heatmap is a good way to understand correlation.**

In [56]:
# FIRST-AUTHOR: remove plotting
# plt.figure(figsize=(13,10))
# sns.heatmap(data.corr(), cmap = "Blues", annot=True, linewidth=3)
_ = data.corr()

/tmp/ipykernel_532876/2339318467.py:4: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  _ = data.corr()


**We can NA_Sales and EU_sales have much effect in Global_Sales. And NA_Sales and EU_sales are mostly corelated**

## Pair plot

In [57]:
data_pair = data.loc[:,["Year","Platform", "Genre", "NA_Sales","EU_Sales", "Other_Sales"]]
data_pair

,Year,Platform,Genre,NA_Sales,EU_Sales,Other_Sales
0,2006.0,Wii,Sports,41.49,29.02,8.46
1,1985.0,NES,Platform,29.08,3.58,0.77
2,2008.0,Wii,Racing,15.85,12.88,3.31
3,2009.0,Wii,Sports,15.75,11.01,2.96
4,1996.0,GB,Role-Playing,11.27,8.89,1.00
...,...,...,...,...,...,...
16593,2002.0,GBA,Platform,0.01,0.00,0.00
16594,2003.0,GC,Shooter,0.01,0.00,0.00
16595,2008.0,PS2,Racing,0.00,0.00,0.00
16596,2010.0,DS,Puzzle,0.00,0.01,0.00


In [58]:
# FIRST-AUTHOR: remove plotting
# sns.pairplot(data_pair, hue='Genre')

# log work

In [59]:
data_pair_log = data_pair.copy()

In [60]:
sale_columns = ['NA_Sales', 'EU_Sales', 'Other_Sales']

In [61]:
# for column in sale_columns:
#     if 0 in data[column].unique():
#         pass
#     else:
#         data_pair_log[column] = np.log(data_pair_log[column])
# #         data_pair_log.head()

In [62]:
data_pair_log = data_pair_log[data_pair_log.NA_Sales != 0]
data_pair_log = data_pair_log[data_pair_log.EU_Sales != 0]
data_pair_log = data_pair_log[data_pair_log.Other_Sales != 0]

In [63]:
data_pair_log

,Year,Platform,Genre,NA_Sales,EU_Sales,Other_Sales
0,2006.0,Wii,Sports,41.49,29.02,8.46
1,1985.0,NES,Platform,29.08,3.58,0.77
2,2008.0,Wii,Racing,15.85,12.88,3.31
3,2009.0,Wii,Sports,15.75,11.01,2.96
4,1996.0,GB,Role-Playing,11.27,8.89,1.00
...,...,...,...,...,...,...
14071,1998.0,PC,Role-Playing,0.01,0.02,0.01
14183,2014.0,PS3,Racing,0.02,0.01,0.01
14185,2011.0,PC,Puzzle,0.01,0.02,0.01
14267,2007.0,PC,Puzzle,0.01,0.02,0.01


In [64]:
data_pair_log['NA_Sales'] = np.log(data_pair_log['NA_Sales']);
data_pair_log['EU_Sales'] = np.log(data_pair_log['EU_Sales']);
data_pair_log['Other_Sales'] = np.log(data_pair_log['Other_Sales']);

In [65]:
# sns.pairplot(data_pair_log, hue='Genre',  diag_kind = 'kde',
#              plot_kws = {'alpha': 0.6, 's': 80, 'edgecolor': 'k'},
#              size = 4)

# FIRST-AUTHOR: remove plotting
# sns.pairplot(data_pair_log, hue='Genre',  palette="husl")

* Me => Ok thats all jaber. are you get all your question answer.
* Jaber => Yah i get, but i have also some questions yet.
* Me => Not today jaber, it's already late night we will do it another day. ok.
* Jaber => Okk.

# Note:
<h4> You already find out that it was just a imaginary story. But truly jaber is real person, he is my room mate also and he loves game too much. I just imagin this story and made it with a little fun. Hope you like it.  </h4>

## If you like this note book please give me an up vote

# Thank You ..............